In [1]:
import joblib
import pandas as pd

from scipy.stats import wilcoxon
from scipy.stats import f_oneway
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn_extra.cluster import KMedoids

In [2]:
k = 3
random_state = 42

In [3]:
dataset = pd.read_csv('../test/data_breast_cancer.csv')
rf = joblib.load(open('../test/random_forest_breat_cancer.joblib','rb'))

In [4]:
X = dataset.loc[:, dataset.columns != 'target']
y = dataset.target

In [5]:
proximity_matrix = X

In [6]:
#proximity_matrix = proximityMatrix(rf, X)
kmedoids = KMedoids(n_clusters=k, random_state=random_state).fit(proximity_matrix)

In [7]:
dataset['cluster'] = kmedoids.labels_
dataset = dataset.sort_values(by='cluster')
dataset

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target,cluster
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,184.6,2019.0,0.1622,0.6656,0.7119,0.26540,0.4601,0.11890,0,0
408,17.99,20.66,117.80,991.7,0.10360,0.13040,0.12010,0.08824,0.1992,0.06069,...,138.1,1349.0,0.1482,0.3735,0.3301,0.19740,0.3060,0.08503,0,0
400,17.91,21.02,124.40,994.0,0.12300,0.25760,0.31890,0.11980,0.2113,0.07115,...,149.6,1304.0,0.1873,0.5917,0.9034,0.19640,0.3245,0.11980,0,0
393,21.61,22.28,144.40,1407.0,0.11670,0.20870,0.28100,0.15620,0.2162,0.06606,...,172.0,2081.0,0.1502,0.5717,0.7053,0.24220,0.3828,0.10070,0,0
392,15.49,19.97,102.40,744.7,0.11600,0.15620,0.18910,0.09113,0.1929,0.06744,...,142.1,1359.0,0.1681,0.3913,0.5553,0.21210,0.3187,0.10190,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,14.80,17.66,95.88,674.8,0.09179,0.08890,0.04069,0.02260,0.1893,0.05886,...,105.9,829.5,0.1226,0.1881,0.2060,0.08308,0.3600,0.07285,1,2
448,14.53,19.34,94.25,659.7,0.08388,0.07800,0.08817,0.02925,0.1473,0.05746,...,108.1,830.5,0.1089,0.2649,0.3779,0.09594,0.2471,0.07463,1,2
370,16.35,23.29,109.00,840.4,0.09742,0.14970,0.18110,0.08773,0.2175,0.06218,...,129.3,1165.0,0.1415,0.4665,0.7087,0.22480,0.4824,0.09614,0,2
199,14.45,20.22,94.49,642.7,0.09872,0.12060,0.11800,0.05980,0.1950,0.06466,...,117.9,1044.0,0.1552,0.4056,0.4967,0.18380,0.4753,0.10130,0,2


In [12]:
import scipy.stats as stats
p_value_table = pd.DataFrame(index=X.columns, columns = ['p_value'])

for feature in X.columns:
    df = dataset[[feature,'cluster']]
    df.columns = ['feature', 'cluster']
    #print(df.head())
    if len(set(df.cluster)) == 2:
        # wilcox test
        wilcoxon_w, wilcoxon_p = wilcoxon(df[df.cluster == 0], df[df.cluster == 1], alternative='two-sided')
    else:
        # anova test
        list_of_df = [df.feature[df.cluster == cluster] for cluster in set(df.cluster)]
        print(stats.f_oneway(*list_of_df))
        model = ols('feature ~ C(cluster)', data=df).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)
        print(anova_table)

F_onewayResult(statistic=1243.5386767947805, pvalue=7.349374997745776e-208)
                 sum_sq     df            F         PR(>F)
C(cluster)  5746.238596    2.0  1243.538677  7.349375e-208
Residual    1307.708038  566.0          NaN            NaN
F_onewayResult(statistic=36.26375223091959, pvalue=1.5178212661423653e-15)
                 sum_sq     df          F        PR(>F)
C(cluster)  1193.486661    2.0  36.263752  1.517821e-15
Residual    9313.893469  566.0        NaN           NaN
F_onewayResult(statistic=1229.9438495905795, pvalue=9.240723309872165e-207)
                   sum_sq     df           F         PR(>F)
C(cluster)  272638.343816    2.0  1229.94385  9.240723e-207
Residual     62731.848552  566.0         NaN            NaN
F_onewayResult(statistic=1032.7002290397918, pvalue=1.3618971793401056e-189)
                  sum_sq     df            F         PR(>F)
C(cluster)  5.521271e+07    2.0  1032.700229  1.361897e-189
Residual    1.513043e+07  566.0          NaN       

In [ ]:
a = ['a','b,'c']
